In [1]:
# Implementatin of tfidf with xg boost 


In [1]:
# Importing libraries

'''
To import any module, run following command in ipynb
import sys
!{sys.executable} -m pip install xgboost
'''

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import xgboost as xgb
import scipy


In [5]:
# Reading data

from ipynb.fs.full.Data_preprocessing import clean

data = pd.read_csv("quora_duplicate_questions.tsv", sep="\t")

data['question1'] = data['question1'].apply(clean)
data['question2'] = data['question2'].apply(clean)

#data.isnull().values.ravel().sum()

In [2]:
# cleaning of the data

# from ipynb.fs.full.Data_preprocessing import clean

#data.isnull().values.ravel().sum()

data = pd.read_csv('CleanData.tsv',sep='\t')
#data1.isnull().values.ravel().sum()
data.describe()
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0


In [6]:
# Splitting the dataset

train_data, test_data = train_test_split(data, test_size = 0.3, random_state = 0)

In [ ]:
#unigram tfidf 

'''
Term Frequency (tf): gives us the frequency of the word in each document in the corpus. 
It is the ratio of number of times the word appears in a document compared to the total 
number of words in that document. It increases as the number of occurrences of that word 
within the document increases. Each document has its own tf.

Inverse Data Frequency (idf): used to calculate the weight of rare words across all documents 
in the corpus. The words that occur rarely in the corpus have a high IDF score.

Combining these two we come up with the TF-IDF score for a word in a document in the corpus.
It is the product of tf and idf:


'''
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit( pd.Series(list(train_data['question1']) + list(train_data['question2'])).astype(str).unique() )
tfidf_train_1 = tfidf_vect.transform(train_data['question1'])
tfidf_train_2 = tfidf_vect.transform(train_data['question2'])
X_train = scipy.sparse.hstack((tfidf_train_1, tfidf_train_2 ))
label1 = train_data['is_duplicate'].values



tfidf_test_1 = tfidf_vect.transform(test_data['question1'])
tfidf_test_2 = tfidf_vect.transform(test_data['question2'])
X_test = scipy.sparse.hstack((tfidf_test_1, tfidf_test_2 ))
label2 = test_data['is_duplicate'].values


xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, label1) 
xgb_prediction = xgb_model.predict(X_test)


In [ ]:
print(accuracy_score(xgb_prediction, label2))

In [ ]:
cm = confusion_matrix(xgb_prediction , label2)
print(cm)

In [ ]:
f1 = f1_score(xgb_prediction , label2, average='macro')
f1

In [ ]:
print(classification_report(xgb_prediction , label2))

In [ ]:
q1 = "Is there anyone here who didn't work too hard and still got into an IIT?"
q2 = "Is there anyone here who didn't work too hard and still got into an IIT?"

q1 = clean(q1)
q2 = clean(q2)

print(q1)
print(q2)

In [ ]:
v1 = tfidf_vect.transform([q1])
v2 = tfidf_vect.transform([q2])

X1= scipy.sparse.hstack((v1, v2 ))
xgb_model.predict(X1)

In [13]:

ntfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
ntfidf_vect.fit( pd.Series(list(train_data['question1']) + list(train_data['question2'])).astype(str).unique() )
ntfidf_train_1 = ntfidf_vect.transform(train_data['question1'])
ntfidf_train_2 = ntfidf_vect.transform(train_data['question2'])
nX_train = scipy.sparse.hstack((ntfidf_train_1, ntfidf_train_2 ))
nlabel1 = train_data['is_duplicate'].values



ntfidf_test_1 = ntfidf_vect.transform(test_data['question1'])
ntfidf_test_2 = ntfidf_vect.transform(test_data['question2'])
nX_test = scipy.sparse.hstack((ntfidf_test_1, ntfidf_test_2 ))
nlabel2 = test_data['is_duplicate'].values


nxgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(nX_train, nlabel1) 
nxgb_prediction = nxgb_model.predict(nX_test)


In [14]:
print(accuracy_score(nxgb_prediction, nlabel2))

0.7362206996627834


In [15]:
ncm = confusion_matrix(nxgb_prediction , nlabel2)
print(ncm)

[[70051 25684]
 [ 6309 19243]]


In [16]:
nf1 = f1_score(nxgb_prediction , nlabel2, average='macro')
nf1

0.6800801157482013

In [17]:
print(classification_report(nxgb_prediction , nlabel2))

              precision    recall  f1-score   support

           0       0.92      0.73      0.81     95735
           1       0.43      0.75      0.55     25552

   micro avg       0.74      0.74      0.74    121287
   macro avg       0.67      0.74      0.68    121287
weighted avg       0.81      0.74      0.76    121287



In [18]:
'''
import pickle
f = open('tfidf_xgb.pickle', 'wb')
pickle.dump(xgb_model, f)
f.close()

f = open('ntfidf_xgb.pickle', 'wb')
pickle.dump(nxgb_model, f)
f.close()

import pickle
f = open('tfidf_vec_xgb.pickle', 'wb')
pickle.dump(tfidf_vect, f)
f.close()

f = open('ntfidf_vec_xgb.pickle', 'wb')
pickle.dump(ntfidf_vect, f)
f.close()
'''




"\nimport pickle\nf = open('tfidf_xgb.pickle', 'wb')\npickle.dump(xgb_model, f)\nf.close()\n\nf = open('ntfidf_xgb.pickle', 'wb')\npickle.dump(nxgb_model, f)\nf.close()\n\nimport pickle\nf = open('tfidf_vec_xgb.pickle', 'wb')\npickle.dump(tfidf_vect, f)\nf.close()\n\nf = open('ntfidf_vec_xgb.pickle', 'wb')\npickle.dump(ntfidf_vect, f)\nf.close()\n"